In [ ]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd

In [ ]:
from keras import backend as K
from keras.engine.topology import Layer
from keras.models import Model
from keras.layers import *

In [ ]:
data=pd.read_csv('/content/drive/My Drive/dataset/IMDB Dataset 2.csv')

In [ ]:
import re

In [ ]:
def preprocess(w):
  w=re.sub(r"([?.!,%&^%$#@])",r" ",w)
  w=re.sub(r'[" "]+'," ",w)
  w=re.sub(r"[^a-zA-Z?.!,]+"," ",w)
  w=w.strip()
  w='<start>'+w+'<end>'
  return w

In [ ]:
data['sentiment'].value_counts()

In [ ]:
texts=list(map(preprocess,data['review']))

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
max_len=256
max_words=20000

In [ ]:
tokenizer=Tokenizer(num_words=10000)
tokenizer.fit_on_texts(texts)
sequences=tokenizer.texts_to_sequences(texts)
word_index=tokenizer.word_index

In [ ]:
data1=pad_sequences(sequences,maxlen=max_len,padding='post')

In [ ]:
data1.shape

(50000, 256)

In [ ]:
labels=data['sentiment']

In [ ]:
labels1=[]

In [ ]:
for i in labels:
  if i=='positive':
    j=1
    labels1.append(j)
  elif i=='negative':
    j=0
    labels1.append(j)

In [ ]:
labels2=np.asarray([labels1])

In [ ]:
labels2=labels2.reshape((50000, ))

In [ ]:
labels2.shape

(50000,)

In [ ]:
train_data=data1[:40000]
train_labels=labels2[:40000]
val_data=data1[40000:]
val_labels=labels2[40000:]

In [ ]:
datasett=tf.data.Dataset.from_tensor_slices((train_data,train_labels))

In [ ]:
datasett3=tf.data.Dataset.from_tensor_slices((data1,labels2))

In [ ]:
final_data=datasett3.batch(64).shuffle(50000)

In [ ]:
datasett2=tf.data.Dataset.from_tensor_slices((val_data,val_labels))

In [ ]:
train_data11=datasett.batch(64).shuffle(40000)

In [ ]:
val_data11=datasett2.batch(64).shuffle(10000)

In [ ]:
next(iter(train_data11))

(<tf.Tensor: shape=(64, 256), dtype=int32, numpy=
 array([[  31,   17,  148, ...,    0,    0,    0],
        [  31, 9533,    6, ...,    0,    0,    0],
        [  22,  111,  235, ...,  130,  658,   24],
        ...,
        [  31,    1,  705, ...,    0,    0,    0],
        [  31,   10,  432, ...,    0,    0,    0],
        [  31,   50,    3, ...,    0,    0,    0]], dtype=int32)>,
 <tf.Tensor: shape=(64,), dtype=int64, numpy=
 array([0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
        0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0])>)

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding,Flatten,Dense,LSTM
from keras import layers
import tensorflow as tf

In [ ]:
def scaled_dot_product_attention(q, k, v, mask=None):
  matmul_qk = tf.matmul(q, k, transpose_b=True)  
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)  
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1) 
  output = tf.matmul(attention_weights, v)  
  return output, attention_weights

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    assert d_model % self.num_heads == 0
    self.depth = d_model // self.num_heads
    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)
    self.dense = tf.keras.layers.Dense(d_model)
  def split_heads(self, x, batch_size):
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])
  def call(self, v, k, q):
    batch_size = tf.shape(q)[0]
    q = self.wq(q)  
    k = self.wk(k)    
    v = self.wv(v)    
    q = self.split_heads(q, batch_size)  
    k = self.split_heads(k, batch_size)  
    v = self.split_heads(v, batch_size) 
    scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v, None)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3]) 
    concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))  
    output = self.dense(concat_attention)  
    return output, attention_weights

In [ ]:
inputt=Input(shape=([None]),dtype='int32')
emb=tf.keras.layers.Embedding(10000,512)(inputt)
_,lstm,__=tf.keras.layers.LSTM(512,return_state=True)(emb)
Out,weight = MultiHeadAttention(512, 8)(lstm,lstm,lstm)
lstm2=tf.keras.layers.LSTM(256)(Out)
den=tf.keras.layers.Dense(128,activation='relu')(lstm2)
drop=tf.keras.layers.Dropout(0.8)(den)
#max=tf.keras.layers.GlobalMaxPool1D()(drop)

den2=tf.keras.layers.Dense(1,activation='sigmoid')(drop)
model=Model(inputs=inputt,outputs=den2)

In [ ]:
model.summary()

Model: "functional_21"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, None, 512)    5120000     input_12[0][0]                   
__________________________________________________________________________________________________
lstm_20 (LSTM)                  [(None, 512), (None, 2099200     embedding_12[0][0]               
__________________________________________________________________________________________________
multi_head_attention_11 (MultiH ((None, None, 512),  1050624     lstm_20[0][1]                    
                                                                 lstm_20[0][1]        

In [ ]:
b=tf.keras.layers.LSTM(256)
dd=tf.keras.layers.Embedding(30000,256)
d=tf.keras.layers.Dense(64)
m=tf.keras.layers.Flatten()
ccc=tf.keras.layers.GlobalMaxPool1D()

In [ ]:
S_inputs = Input(shape=([None]), dtype='int32')
embeddings = dd(S_inputs)

c=b(embeddings)
O_seq,a = MultiHeadAttention(512, 8)(c,c,c)
b_seq=d(O_seq)
e = Dropout(0)(b_seq)
e=ccc(e)
outputs = Dense(1, activation='softmax')(e)
model1 = Model(inputs=S_inputs, outputs=outputs)

In [ ]:
model2= tf.keras.Sequential([
    tf.keras.layers.Embedding(10000, 128),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [ ]:
from keras import optimizers

In [ ]:
loss_fn=tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer=tf.keras.optimizers.SGD(learning_rate=0.1), metrics=['accuracy'])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001,beta_1=0.9,beta_2=0.9999,epsilon=None,decay=0.01,amsgrad=False), metrics=['accuracy'])

In [ ]:
hist= model.fit(final_data,epochs=30)

In [ ]:
hist= model.fit(train_data11,epochs=20,validation_data=(val_data11))